In [1]:
import os
os.environ["SWI_HOME_DIR"] = os.path.expanduser("~/.local/swipl/lib/swipl")
os.environ["LD_LIBRARY_PATH"] = os.path.expanduser("~/.local/swipl/lib/swipl/lib/x86_64-linux")
os.environ["PATH"] = os.path.expanduser("~/.local/swipl/bin") + ":" + os.environ["PATH"]

from pyswip import Prolog
from pathlib import Path
from deap import base, creator, tools, algorithms
import numpy as np
import deap 
import random


In [2]:
base_dir = Path("../symbolic_reasoning_sys/expanded versions/")
PROLOG_FILE = base_dir / "ethics_engine_expanded_64.pl"
prolog = Prolog()
prolog.consult(PROLOG_FILE)

# Call the main decision function
query = list(prolog.query("make_decision(dropped_wallet_32, Action, Justification, Score)"))
for result in query:
    print(f"Action: {result['Action']}")
    print(f"Justification: {result['Justification']}")
    print(f"Score: {result['Score']}")

Action: leave_wallet
Justification: Action leave_wallet justified by utilitarian ethics based on scenario context: leave_wallet.
Score: 0.3


In [3]:
GROUND_TRUTH = {
    "dropped_wallet_1": "return_wallet",
    "dropped_wallet_2": "take_wallet", 
    "dropped_wallet_3": "return_wallet",
    "dropped_wallet_4": "leave_wallet",
    "dropped_wallet_5": "return_wallet",
    "dropped_wallet_6": "leave_wallet",
    "dropped_wallet_7": "return_wallet",
    "dropped_wallet_8": "take_wallet",
    "dropped_wallet_9": "return_wallet",
    "dropped_wallet_10": "take_wallet", 
    "dropped_wallet_11": "return_wallet",
    "dropped_wallet_12": "leave_wallet", 
    "dropped_wallet_13": "return_wallet",
    "dropped_wallet_14": "leave_wallet",
    "dropped_wallet_15": "return_wallet",
    "dropped_wallet_16": "take_wallet",
    "dropped_wallet_17": "return_wallet",
    "dropped_wallet_18": "leave_wallet", 
    "dropped_wallet_19": "return_wallet",
    "dropped_wallet_20": "leave_wallet",
    "dropped_wallet_21": "return_wallet",
    "dropped_wallet_22": "leave_wallet",
    "dropped_wallet_23": "return_wallet",
    "dropped_wallet_24": "leave_wallet",
    "dropped_wallet_25": "return_wallet",
    "dropped_wallet_26": "leave_wallet", 
    "dropped_wallet_27": "return_wallet",
    "dropped_wallet_28": "leave_wallet",
    "dropped_wallet_29": "return_wallet",
    "dropped_wallet_30": "leave_wallet",
    "dropped_wallet_31": "return_wallet",
    "dropped_wallet_32": "leave_wallet",
    "dropped_wallet_33": "leave_wallet",
    "dropped_wallet_34": "take_wallet",
    "dropped_wallet_35": "leave_wallet",
    "dropped_wallet_36": "leave_wallet",
    "dropped_wallet_37": "leave_wallet",
    "dropped_wallet_38": "leave_wallet",
    "dropped_wallet_39": "leave_wallet",
    "dropped_wallet_40": "take_wallet",
    "dropped_wallet_41": "take_wallet",
    "dropped_wallet_42": "take_wallet",
    "dropped_wallet_43": "leave_wallet",
    "dropped_wallet_44": "leave_wallet",
    "dropped_wallet_45": "take_wallet",
    "dropped_wallet_46": "take_wallet",
    "dropped_wallet_47": "take_wallet",
    "dropped_wallet_48": "take_wallet",
    "dropped_wallet_49": "leave_wallet", 
    "dropped_wallet_50": "leave_wallet",
    "dropped_wallet_51": "leave_wallet",
    "dropped_wallet_52": "leave_wallet",
    "dropped_wallet_53": "leave_wallet",
    "dropped_wallet_54": "leave_wallet",
    "dropped_wallet_55": "leave_wallet",
    "dropped_wallet_56": "leave_wallet",
    "dropped_wallet_57": "leave_wallet",
    "dropped_wallet_58": "leave_wallet",
    "dropped_wallet_59": "leave_wallet",  
    "dropped_wallet_60": "leave_wallet",
    "dropped_wallet_61": "leave_wallet",
    "dropped_wallet_62": "leave_wallet",
    "dropped_wallet_63": "leave_wallet",
    "dropped_wallet_64": "leave_wallet",
}


In [4]:
def update_weights_in_file(weights):
    """
    Overwrite weight/2 facts in the Prolog file with new values.
    weights is a list or array: [w_utilitarian, w_deontological, w_self_interest]
    """
    with open(PROLOG_FILE, "r", encoding="utf-8") as f:
        lines = f.readlines()
    new_lines = []
    for line in lines:
        if line.strip().startswith("weight(utilitarian"):
            new_lines.append(f"weight(utilitarian, {weights[0]:.3f}).\n")
        elif line.strip().startswith("weight(deontological"):
            new_lines.append(f"weight(deontological, {weights[1]:.3f}).\n")
        elif line.strip().startswith("weight(self_interest"):
            new_lines.append(f"weight(self_interest, {weights[2]:.3f}).\n")
        else:
            new_lines.append(line)

    with open(PROLOG_FILE, "w", encoding="utf-8") as f:
        f.writelines(new_lines)
        

In [5]:
def run_prolog_query(weights):
    """
    Given a weight vector, update Prolog file, reload Prolog engine,
    run make_decision for all scenarios and calculate accuracy.
    """
    
    weights = np.array(weights)
    weights = weights / np.sum(weights) # Normalise weights so they sum to 1

    update_weights_in_file(weights)
    prolog = Prolog()
    prolog.consult(PROLOG_FILE)
    # for fact in prolog.query("weight(X, V)"):
    #     print(fact)


    correct = 0
    total = len(GROUND_TRUTH)

    for scenario_name, true_action in GROUND_TRUTH.items():
        query = f"make_decision({scenario_name}, Action, Justification, Score)."
        results = list(prolog.query(query))
        if not results:
            print(f"No result for scenario {scenario_name}")
            continue
        predicted = results[0]["Action"]
        if predicted == true_action:
            correct += 1

    accuracy = correct / total
    return (accuracy,)

if __name__ == "__main__":
    test_weights = [0.33, 0.33, 0.34]
    acc = run_prolog_query(test_weights)[0]
    print(f"Accuracy for weights {test_weights}: {acc:.2f}")
    

Accuracy for weights [0.33, 0.33, 0.34]: 0.62


In [6]:
scenario_data = [
    ("dropped_wallet_1", ("dropped_wallet", True, True, "many_people_around", "none", True)),
    ("dropped_wallet_2", ("dropped_wallet", True, True, "many_people_around", "none", False)),
    ("dropped_wallet_3", ("dropped_wallet", True, True, "many_people_around", "cctv_visible", True)),
    ("dropped_wallet_4", ("dropped_wallet", True, True, "many_people_around", "cctv_visible", False)),
    ("dropped_wallet_5", ("dropped_wallet", True, True, "many_people_around", "law_posted", True)),
    ("dropped_wallet_6", ("dropped_wallet", True, True, "many_people_around", "law_posted", False)),
    ("dropped_wallet_7", ("dropped_wallet", True, True, "many_people_around", "unclear", True)),
    ("dropped_wallet_8", ("dropped_wallet", True, True, "many_people_around", "unclear", False)),
    ("dropped_wallet_9", ("dropped_wallet", True, True, "isolated_area", "none", True)),
    ("dropped_wallet_10", ("dropped_wallet", True, True, "isolated_area", "none", False)),
    ("dropped_wallet_11", ("dropped_wallet", True, True, "isolated_area", "cctv_visible", True)),
    ("dropped_wallet_12", ("dropped_wallet", True, True, "isolated_area", "cctv_visible", False)),
    ("dropped_wallet_13", ("dropped_wallet", True, True, "isolated_area", "law_posted", True)),
    ("dropped_wallet_14", ("dropped_wallet", True, True, "isolated_area", "law_posted", False)),
    ("dropped_wallet_15", ("dropped_wallet", True, True, "isolated_area", "unclear", True)),
    ("dropped_wallet_16", ("dropped_wallet", True, True, "isolated_area", "unclear", False)),
    ("dropped_wallet_17", ("dropped_wallet", True, False, "many_people_around", "none", True)),
    ("dropped_wallet_18", ("dropped_wallet", True, False, "many_people_around", "none", False)),
    ("dropped_wallet_19", ("dropped_wallet", True, False, "many_people_around", "cctv_visible", True)),
    ("dropped_wallet_20", ("dropped_wallet", True, False, "many_people_around", "cctv_visible", False)),
    ("dropped_wallet_21", ("dropped_wallet", True, False, "many_people_around", "law_posted", True)),
    ("dropped_wallet_22", ("dropped_wallet", True, False, "many_people_around", "law_posted", False)),
    ("dropped_wallet_23", ("dropped_wallet", True, False, "many_people_around", "unclear", True)),
    ("dropped_wallet_24", ("dropped_wallet", True, False, "many_people_around", "unclear", False)),
    ("dropped_wallet_25", ("dropped_wallet", True, False, "isolated_area", "none", True)),
    ("dropped_wallet_26", ("dropped_wallet", True, False, "isolated_area", "none", False)),
    ("dropped_wallet_27", ("dropped_wallet", True, False, "isolated_area", "cctv_visible", True)),
    ("dropped_wallet_28", ("dropped_wallet", True, False, "isolated_area", "cctv_visible", False)),
    ("dropped_wallet_29", ("dropped_wallet", True, False, "isolated_area", "law_posted", True)),
    ("dropped_wallet_30", ("dropped_wallet", True, False, "isolated_area", "law_posted", False)),
    ("dropped_wallet_31", ("dropped_wallet", True, False, "isolated_area", "unclear", True)),
    ("dropped_wallet_32", ("dropped_wallet", True, False, "isolated_area", "unclear", False)),
    ("dropped_wallet_33", ("dropped_wallet", False, True, "many_people_around", "none", True)),
    ("dropped_wallet_34", ("dropped_wallet", False, True, "many_people_around", "none", False)),
    ("dropped_wallet_35", ("dropped_wallet", False, True, "many_people_around", "cctv_visible", True)),
    ("dropped_wallet_36", ("dropped_wallet", False, True, "many_people_around", "cctv_visible", False)),
    ("dropped_wallet_37", ("dropped_wallet", False, True, "many_people_around", "law_posted", True)),
    ("dropped_wallet_38", ("dropped_wallet", False, True, "many_people_around", "law_posted", False)),
    ("dropped_wallet_39", ("dropped_wallet", False, True, "many_people_around", "unclear", True)),
    ("dropped_wallet_40", ("dropped_wallet", False, True, "many_people_around", "unclear", False)),
    ("dropped_wallet_41", ("dropped_wallet", False, True, "isolated_area", "none", True)),
    ("dropped_wallet_42", ("dropped_wallet", False, True, "isolated_area", "none", False)),
    ("dropped_wallet_43", ("dropped_wallet", False, True, "isolated_area", "cctv_visible", True)),
    ("dropped_wallet_44", ("dropped_wallet", False, True, "isolated_area", "cctv_visible", False)),
    ("dropped_wallet_45", ("dropped_wallet", False, True, "isolated_area", "law_posted", True)),
    ("dropped_wallet_46", ("dropped_wallet", False, True, "isolated_area", "law_posted", False)),
    ("dropped_wallet_47", ("dropped_wallet", False, True, "isolated_area", "unclear", True)),
    ("dropped_wallet_48", ("dropped_wallet", False, True, "isolated_area", "unclear", False)),
    ("dropped_wallet_49", ("dropped_wallet", False, False, "many_people_around", "none", True)),
    ("dropped_wallet_50", ("dropped_wallet", False, False, "many_people_around", "none", False)),
    ("dropped_wallet_51", ("dropped_wallet", False, False, "many_people_around", "cctv_visible", True)),
    ("dropped_wallet_52", ("dropped_wallet", False, False, "many_people_around", "cctv_visible", False)),
    ("dropped_wallet_53", ("dropped_wallet", False, False, "many_people_around", "law_posted", True)),
    ("dropped_wallet_54", ("dropped_wallet", False, False, "many_people_around", "law_posted", False)),
    ("dropped_wallet_55", ("dropped_wallet", False, False, "many_people_around", "unclear", True)),
    ("dropped_wallet_56", ("dropped_wallet", False, False, "many_people_around", "unclear", False)),
    ("dropped_wallet_57", ("dropped_wallet", False, False, "isolated_area", "none", True)),
    ("dropped_wallet_58", ("dropped_wallet", False, False, "isolated_area", "none", False)),
    ("dropped_wallet_59", ("dropped_wallet", False, False, "isolated_area", "cctv_visible", True)),
    ("dropped_wallet_60", ("dropped_wallet", False, False, "isolated_area", "cctv_visible", False)),
    ("dropped_wallet_61", ("dropped_wallet", False, False, "isolated_area", "law_posted", True)),
    ("dropped_wallet_62", ("dropped_wallet", False, False, "isolated_area", "law_posted", False)),
    ("dropped_wallet_63", ("dropped_wallet", False, False, "isolated_area", "unclear", True)),
    ("dropped_wallet_64", ("dropped_wallet", False, False, "isolated_area", "unclear", False)),
]


all_scenarios = {
    name: (event, (owner_nearby, contents_valuable, environment, legal_context, owner_traceability))
    for name, (event, owner_nearby, contents_valuable, environment, legal_context, owner_traceability) in scenario_data
}

def predict_action(scenario_name, weights):
    update_weights_in_file(weights)
    result = list(prolog.query(f"make_decision({scenario_name}, Action, Justification, Score)"))
    if result:
        predicted = result[0]["Action"]
        return predicted, result[0]["Justification"], result[0]["Score"]
    else:
        print(f"Scenario: {scenario_name} - No decision returned")
        return None, None, None

def evaluate(weights):
    weights = np.array(weights)
    weights = np.abs(weights)  # Ensure positive weights
    weights = weights / np.sum(weights)  # Normalize
    return run_prolog_query(weights)
    
def check_valid(individual):
    """Ensure weights are positive"""
    return all(w >= 0 for w in individual)
    
# === DEAP SETUP ===

if not hasattr(creator, "FitnessMax"):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if not hasattr(creator, "Individual"):
    creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.uniform, 0, 1) # Individuals are 3 floating-point numbers between 0 and 1
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.1)  # Less disruptive crossover
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

toolbox.decorate("mate", tools.DeltaPenalty(check_valid, 0.1))
toolbox.decorate("mutate", tools.DeltaPenalty(check_valid, 0.1))


    
def main():
    random.seed(42)
    population = toolbox.population(n=10)
    NGEN = 20
    CXPB = 0.7  # crossover prob
    MUTPB = 0.2  # mutation prob

    print("Start of evolution")
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    for gen in range(1, NGEN + 1):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values, child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate invalid fitnesses
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid))
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit

        population[:] = offspring
        ###
        print(f"\nGeneration {gen} - Population Weights:")
        for i, ind in enumerate(population):
            normalised = np.round(np.array(ind) / np.sum(ind), 3)
            print(f"  Ind {i}: {normalised} (Fitness: {ind.fitness.values[0]:.3f})")
        ###
        top = tools.selBest(population, 1)[0]
        normalised_weights = np.round(np.array(top) / np.sum(top), 3)
        print(f"Gen {gen}: Best Accuracy = {top.fitness.values[0]:.2f} | Weights = {normalised_weights}")

    best_ind = tools.selBest(population, 1)[0]
    print("\nBest individual:")
    normalised_weights = np.round(np.array(best_ind) / np.sum(best_ind), 3)
    print(f"Weights: Utilitarian={normalised_weights[0]:.3f}, Deontological={normalised_weights[1]:.3f}, Self-interest={normalised_weights[2]:.3f}")
    print(f"Accuracy: {best_ind.fitness.values[0]:.3f}")
    print(best_ind)
    for scenario in all_scenarios:
        action, justification, score = predict_action(scenario, best_ind)
        correct = GROUND_TRUTH.get(scenario)
        scenario_struct_result = list(prolog.query(f"scenario({scenario}, S)"))
        if not scenario_struct_result:
            print(f"Scenario: {scenario} --- No scenario struct found")
            continue
        scenario_struct = scenario_struct_result[0]["S"]
        rules_result = list(prolog.query(f"rule_sources({repr(scenario_struct)}, {action}, Sources)"))
        rules = rules_result[0]["Sources"] if rules_result else []
    
        match = action == correct
        print(f"Scenario: {scenario} --- Predicted: {action}, justification: {justification}, score: {score}, Ground Truth: {correct}, Match: {'✅' if match else '❌'}{match}")

if __name__ == "__main__":
    main()

Start of evolution

Generation 1 - Population Weights:
  Ind 0: [0.4   0.157 0.442] (Fitness: 0.562)
  Ind 1: [0.291 0.388 0.321] (Fitness: 0.719)
  Ind 2: [0.062 0.547 0.39 ] (Fitness: 0.719)
  Ind 3: [0.062 0.547 0.39 ] (Fitness: 0.719)
  Ind 4: [0.136 0.45  0.414] (Fitness: 0.719)
  Ind 5: [0.134 0.451 0.415] (Fitness: 0.719)
  Ind 6: [0.136 0.45  0.414] (Fitness: 0.719)
  Ind 7: [0.402 0.163 0.435] (Fitness: 0.562)
  Ind 8: [0.019 0.538 0.443] (Fitness: 0.719)
  Ind 9: [0.222 0.215 0.563] (Fitness: 0.562)
Gen 1: Best Accuracy = 0.72 | Weights = [0.291 0.388 0.321]

Generation 2 - Population Weights:
  Ind 0: [0.096 0.49  0.414] (Fitness: 0.719)
  Ind 1: [0.236 0.452 0.312] (Fitness: 0.719)
  Ind 2: [0.148 0.487 0.365] (Fitness: 0.719)
  Ind 3: [0.019 0.538 0.443] (Fitness: 0.719)
  Ind 4: [0.321 0.436 0.243] (Fitness: 0.719)
  Ind 5: [0.126 0.439 0.435] (Fitness: 0.719)
  Ind 6: [0.136 0.45  0.414] (Fitness: 0.719)
  Ind 7: [0.136 0.45  0.414] (Fitness: 0.719)
  Ind 8: [0.019 0.538

In [7]:
print(run_prolog_query([0.8, 0.1, 0.1]))  # Mostly utilitarian
print(run_prolog_query([0.1, 0.8, 0.1]))  # Mostly deontological
print(run_prolog_query([0.1, 0.1, 0.8]))  # Mostly self-interest
for w in [[1, 0, 0], [0, 1, 0], [0, 0, 1], [0.5, 0.5, 0], [0.3, 0.3, 0.4]]:
    print(w, run_prolog_query(w))

(0.546875,)
(0.71875,)
(0.625,)
[1, 0, 0] (0.546875,)
[0, 1, 0] (0.71875,)
[0, 0, 1] (0.625,)
[0.5, 0.5, 0] (0.671875,)
[0.3, 0.3, 0.4] (0.625,)


In [8]:
(32-7)/32

0.78125